In [31]:
# TODO need to account for transfers so they don't mess up the numbers

# TODO more risk features: hour of day, *** roc of 12h / 24h avg med_spread ***

# TODO cashout strategy - check account balance daily, if balance has gone up, transfer half of the increase to funding wallet. when i'm ready to implement the auto cash-out, i should call the function right before top-up bnb gets called on the daily open session. then i can check if the daily pnl was greater than 10 usdt and if there is more than half of that pnl in free bnb and more than {top-up bnb amount} in free usdt. if all those are true, just transfer out that amount of bnb, if there is enough pnl and half the pnl in usdt but not bnb, then buy the bnb and transfer it out, else, do nothing. and in all cases, remember to update asset balances afterwards

In [32]:
import pandas as pd
from pathlib import Path
import json
from datetime import datetime, timedelta
import plotly.graph_objects as go

pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 4)

In [39]:
session_log_path = Path("/home/ross/coding/modular_trader/records/session_log.json")
with open(session_log_path, "r") as f:
    session_log = json.load(f)
balances = [{'timestamp': sl['timestamp'],
             'med_spread': sl['median_spread'],
             'avg_open_size': sl['avg_open_size'],
             'max_open_size': sl['max_open_size'],
             'spot_balance': sl['spot_balance'],
             'margin_balance': sl['margin_balance']}
            for sl in session_log]
df1 = pd.DataFrame(balances)
df1['timestamp'] = pd.to_datetime(df1.timestamp, format="%d/%m/%y %H:%M")
print(len(df1))

2897


In [40]:
agent_log_path = Path("/home/ross/coding/modular_trader/records/channel_run_15m_None_200_edge_mid_volume_1d/perf_log.json")
with open(agent_log_path, "r") as f:
    agent_log = json.load(f)
agent_bals = [{'timestamp': sl['timestamp'],
              'med_spread': sl.get('median_spread', 0),
              'avg_open_size': sl.get('avg_open_size', 0),
              'max_open_size': sl.get('max_open_size', 0),
              'margin_balance': sl.get('balance', 0)}
             for sl in agent_log[:900]]
df2 = pd.DataFrame(agent_bals)
df2['timestamp'] = pd.to_datetime(df2.timestamp, format="%d/%m/%y %H:%M")
df2 = df2.loc[df2.timestamp < df1.timestamp.iloc[0]]
print(len(df2))

867


In [41]:
df = pd.concat([df2, df1]).set_index('timestamp', drop=True)
df_mean = df.resample('H').agg('mean')

window = datetime.now() - timedelta(weeks=2)
df = df.loc[df.index > window]
df_mean = df_mean.loc[df_mean.index > window]
print(len(df))
print(len(df_mean))

783
336


In [42]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=df_mean.index, y=df_mean.margin_balance.interpolate(method='linear').rolling(4).mean(), name='margin_balance', mode="lines", line=go.scatter.Line(color="#1f77b4")))
# fig.add_trace(go.Scatter(x=df.index, y=df.avg_open_size.rolling(16).mean(), name='avg_open_size', yaxis="y2", mode="lines", line=go.scatter.Line(color="#ff7f0e")))
fig.add_trace(go.Scatter(x=df_mean.index, y=df_mean.med_spread.interpolate(method='linear').rolling(12).mean(), name='median_spread', yaxis="y3", mode="lines", line=go.scatter.Line(color="#d62728")))

# Create axis objects
fig.update_layout(xaxis=dict(domain=[0.06, 1]), 
                  yaxis=dict(title='margin_balance', titlefont=dict(color="#1f77b4"), tickfont=dict(color="#1f77b4")), 
                  # yaxis2=dict(title='avg_open_size', titlefont=dict(color="#ff7f0e"), tickfont=dict(color="#ff7f0e"), 
                  #             anchor="free", overlaying="y", side="left", position=0.06), 
                  yaxis3=dict(title='median_spread', titlefont=dict(color="#d62728"), tickfont=dict(color="#d62728"), 
                              anchor="free", overlaying="y", side="right", position=1))

# Update layout properties
fig.update_layout(
    title_text=f"Current Margin Balance: {df_mean.margin_balance.iloc[-1]:.2f} USDT",
    title_font_size=30,
    width=1200, height=600,
    margin=dict(l=50, r=25, b=25, t=50),
    legend=dict(orientation="h", 
                xref='paper', x=0.5, xanchor="center",
                yref="paper", y=1, yanchor="top")
)

fig.show()